## 01 | Seasonality & Pricing Trends: Monthly ADR Performance by Hotel Type

**Business Question:** How does the Average Daily Rate (ADR) fluctuate throughout the year across different hotel types, and when do the actual peak seasons occur?

In [ ]:
WITH monthly_pricing_data AS (
	SELECT 
		hb.hotel,
		hb.arrival_month,
		EXTRACT(MONTH FROM hb.arrival_date) AS month_num,
		hb.adr
	FROM hotel_bookings hb
	WHERE hb.is_canceled = 0
)
SELECT 
	mpd.hotel,
	mpd.arrival_month,
	COUNT(*) AS total_bookings,
	ROUND(AVG(mpd.adr)::NUMERIC, 2) AS avg_adr
FROM monthly_pricing_data mpd
GROUP BY mpd.hotel, mpd.arrival_month, mpd.month_num
ORDER BY mpd.month_num;

### Visual Insight: ADR Seasonality & Hotel Comparison

This visualization examines the fluctuations in Average Daily Rate (ADR) throughout the year, comparing the pricing strategies between City Hotels and Resort Hotels. Identifying these peaks is crucial for defining the actual high season and optimizing dynamic pricing models.

![ADR Seasonality Comparison](../images/financial_analysis/adr_seasonality_hotel_type.PNG)

**Business Analysis:** The data reveals a stark contrast in revenue volatility between hotel types. The **Resort Hotel** exhibits extreme seasonality, with prices peaking in **August at $182.10**, followed by a sharp decline to a minimum of **$48.96 in November**. Conversely, the **City Hotel** maintains a remarkably stable ADR throughout the year, suggesting a more consistent demand. To mitigate the resort's off-season revenue drop, it is recommended to implement aggressive marketing campaigns for Q4, targeting segments with flexible travel schedules or corporate events to improve occupancy during low-rate months.

## 02 | Customer Segment Profitability: Family vs. Adults-Only ADR Analysis

**Business Question:** Do bookings with children or babies generate a higher Average Daily Rate (ADR) that justifies the potential increase in operational costs?

In [ ]:
WITH guest_composition_adr AS (
	SELECT
		hb.hotel,
		CASE
			WHEN hb.children > 0 OR hb.babies > 0 THEN 'Family'
			ELSE 'Adults Only'
		END AS guest_type,
		hb.adr
	FROM hotel_bookings hb 
	WHERE hb.is_canceled = 0
)
SELECT
	gca.hotel,
	gca.guest_type,
	COUNT(*) AS total_bookings,
	ROUND(AVG(gca.adr)::NUMERIC, 2) AS avg_adr
FROM guest_composition_adr gca
GROUP BY gca.hotel, gca.guest_type
ORDER BY gca.hotel, avg_adr DESC;

### Visual Insight: Profitability Analysis by Guest Composition

This analysis compares the Average Daily Rate (ADR) between families (bookings with children or babies) and adults-only reservations. By evaluating these segments, the hotel can determine if the operational costs associated with family-friendly amenities are justified by a higher revenue per room.

![Guest Composition Profitability](../images/financial_analysis/adr_family_vs_adults.PNG)

**Business Analysis:** The data confirms that the **Family** segment is a high-value driver for the hotel, with an ADR of **$150.82** compared to **$97.31** for **Adults Only** bookings. This represents a substantial **54.98% premium** in daily revenue, which effectively offsets the higher operational costs and specialized amenities required for children. Given this significant margin, the hotel should prioritize marketing strategies and infrastructure investments—such as kids' clubs or family-oriented entertainment—to further attract this lucrative segment and maximize RevPAR (Revenue Per Available Room).

## 03 | Pricing Benchmark: Market Segment ADR vs. Global Average

**Business Question:** Which market segments are performing above or below the hotel's global average ADR, and where is there potential for price optimization?

In [ ]:
WITH segment_performance_metrics AS (
	SELECT
		hb.market_segment,
		SUM(hb.adr) AS total_segment_revenue,
		COUNT(*) AS segment_bookings,
		ROUND(AVG(hb.adr)::NUMERIC, 2) AS segment_avg_adr
	FROM hotel_bookings hb 
	WHERE 
		hb.is_canceled = 0
		AND hb.market_segment <> 'Undefined'
	GROUP BY hb.market_segment
)
SELECT
	spm.market_segment,
	spm.segment_avg_adr,
	ROUND(
		(SUM(spm.total_segment_revenue) OVER () / SUM(spm.segment_bookings) OVER ())::NUMERIC, 
		2
	) AS global_weighted_adr,
	ROUND(
		(spm.segment_avg_adr - (SUM(spm.total_segment_revenue) OVER () / SUM(spm.segment_bookings) OVER ()))::NUMERIC, 
		2
	) AS diff_vs_global
FROM segment_performance_metrics spm
ORDER BY spm.segment_avg_adr DESC;

### Visual Insight: Market Segment Pricing Benchmark

This analysis utilizes window functions to compare the Average Daily Rate (ADR) of each market segment against the hotel's global weighted average. This comparison is vital for identifying "underpriced" segments and adjusting negotiation strategies with travel agents and corporate partners.

![Market Segment Benchmark](../images/financial_analysis/adr_market_segment_benchmark.PNG)

**Business Analysis:** The benchmark reveals that **Direct ($115.22)** and **Online TA ($114.92)** are the primary revenue drivers, consistently performing above the **Global Weighted ADR of $102.27**. While segments like **Groups** and **Corporate** fall below this average, this often reflects pre-negotiated volume discounts rather than underpricing. Most notably, the **Complementary** segment sits **96.79% below** the global average at just **$3.28**; while expected for non-revenue bookings, its volume should be strictly monitored to avoid displacing high-value guests. For underperforming commercial segments, the hotel should review contract terms and focus on "upselling" additional services to compensate for the lower daily rates.